# Exposure age calculation

## Context

A <sup>10</sup>Be concentration of 100 x 10<sup>3</sup> at/g characterizes a shallow bedrock sample, if we consider granite, density = 2.7). The production rate at sea level high latitude (SLHL) is 1 at/g/yr. <sup>10</sup>Be half live is 1.37 x 10<sup>6</sup> yr. 

The bedrock is characterized with:

## Question 1

Calculate the exposure age for an erosion rate of 1, 2, 5, 10, and 50 m/Myr. How is the exposure age evolving in function of the erosion rate ? Why ?

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.express as px
import os, sys, platform

# Define root directory according to current environment
if platform.node() != "se22.156":
    !git clone https://github.com/franz825/cosmo-training
    dir_root = "/content/cosmo-training"

else:
    # Path to root directory (repository)
    dir_root = os.path.dirname(sys.path[0])

# Import custom functions for CRN computation
sys.path.append(dir_root)
from src.functions.surface_concentration import surface_concentration


Define variables

In [ ]:
# Constant erosion rate of the surface (cm/yr)
erosion_rates = [0.000001, 0.0001, 0.001, 0.03]

# Vector containing the exposure ages (yr) to process
ages = [1000, 10000, 20000, 30000, 50000, 100000, 200000, 300000, 400000, 500000, 1000000]    

In [ ]:
# Containers for 10Be concentrations results
crn_table = pd.DataFrame({"exposure_age": ages})
crn_plot = pd.DataFrame()

# Loop within erosion rate values
for i in range(len(erosion_rates)):

    # Get current erosion rate
    erosion_rate = erosion_rates[i]

    # Array to contain CRN concentration for each 
    crn_concentrations = np.zeros(len(ages))

    # Loop within exposure age values
    for i in range(len(ages)): 
        
        # Get current exposure age
        exposure_age = ages[i]

        # Compute 10Be concentration at the surface 
        Be_concentration = conc_surf(erosion_rate, exposure_age)

        # Divide 10Be concentration by 1000 (and store in array
        crn_concentrations[i] = Be_concentration

    # Convert array of concentrations in a named serie
    crn_concentrations = pd.Series(crn_concentrations, name = "crn_conc_erate_" + str(erosion_rate), dtype = 'Float64')
    # Include array of concentrations in a complete dataframe
    crn_df = pd.DataFrame({"exposure_age": ages, "crn_concentration": crn_concentrations, "erosion_rate": pd.Series(np.repeat(erosion_rate, len(ages)))})

    # Collect concentrations formatted for plots
    crn_plot = pd.concat([crn_plot, crn_df], axis = 0)
    # Collect concentrations into pandas dataframe for csv export
    crn_table = pd.concat([crn_table, crn_concentrations], axis = 1)

Graph

In [ ]:
# Write results to csv file
crn_table.to_csv("exposure_ages.csv")

# Print the table containing 10Be concentration for each exposure age
display(crn_table)

In [ ]:
# Coerce erosion rate as string and convert at/g to 10^3 at/g
crn_plot["erosion_rate"] = crn_plot["erosion_rate"] * 10000
crn_plot["erosion_rate"] = crn_plot["erosion_rate"].astype(str)
crn_plot["crn_concentration"] = crn_plot["crn_concentration"] / 1000

# Plot 10Be concentrations for various erosion rates 
plot = px.scatter(crn_plot, x = "exposure_age", y = "crn_concentration", color = "erosion_rate", labels = dict(exposure_age = "Exposure age (a)", 
crn_concentration = "Concentration (10<sup>3</sup> at/g)", erosion_rate = "Erosion rate (m /Ma)"), log_x = True, log_y = True)
plot.show()